#### Dataset: CIFAR-10 

##### Cifar10 image classification using customized Net architecture with data augmentation

This code sets up a deep learning pipeline using PyTorch to train a Convolutional Neural Network (CNN) on the CIFAR-10 dataset, a standard benchmark in computer vision. It begins by importing necessary PyTorch modules, setting up data augmentation techniques for training and testing datasets, and preparing the data loaders. This is crucial for enhancing the model's generalization by introducing variability in the training data, thus preventing overfitting.

Then further is demonstrated how to modify a pre-trained ResNet18 model to classify images into 10 categories specific to the CIFAR-10 dataset. By freezing the pre-existing layers of the model and replacing the final fully connected layer to match the number of target classes, the snippet efficiently leverages transfer learning. This approach significantly reduces training time and computational expense by reusing learned features from a model trained on a much larger dataset. The training loop involves forward passes, loss computation, backward passes for gradient computation, and optimizer steps to update the model's weights, illustrating the entire process of training a neural network in PyTorch.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

# Set up data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Load the pre-trained ResNet18 model
resnet = models.resnet18(pretrained=True)

# Freeze all the parameters in the pre-trained model
for param in resnet.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a new one that has 10 outputs
resnet.fc = nn.Linear(512, 10)

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)

# Set up the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.fc.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# Train the model for 20 epochs
import time

start_time = time.time()
for epoch in range(20):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = resnet(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update the weights
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')
print('Finished training in %.3f seconds' % (time.time() - start_time))


Files already downloaded and verified
Files already downloaded and verified


c:\Users\mimis\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mimis\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[1,   100] loss: 14.990
[1,   200] loss: 12.055
[1,   300] loss: 11.857
[1,   400] loss: 12.062
[1,   500] loss: 12.149
[1,   600] loss: 13.187
[1,   700] loss: 13.664
[2,   100] loss: 13.116
[2,   200] loss: 13.599
[2,   300] loss: 13.004
[2,   400] loss: 12.232
[2,   500] loss: 12.206
[2,   600] loss: 12.166
[2,   700] loss: 12.395
[3,   100] loss: 13.371
[3,   200] loss: 12.793
[3,   300] loss: 12.809
[3,   400] loss: 11.556
[3,   500] loss: 11.588
[3,   600] loss: 12.665
[3,   700] loss: 12.897
[4,   100] loss: 13.905
[4,   200] loss: 13.187
[4,   300] loss: 12.485
[4,   400] loss: 12.509
[4,   500] loss: 11.955
[4,   600] loss: 13.491
[4,   700] loss: 13.210
[5,   100] loss: 12.467
[5,   200] loss: 12.940
[5,   300] loss: 13.305
[5,   400] loss: 13.378
[5,   500] loss: 13.946
[5,   600] loss: 13.061
[5,   700] loss: 12.250
[6,   100] loss: 12.589
[6,   200] loss: 11.912
[6,   300] loss: 12.813
[6,   400] loss: 12.782
[6,   500] loss: 11.498
[6,   600] loss: 12.026
[6,   700] loss:

In [5]:
# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
print('Finished testing in %.3f seconds' % (time.time() - start_time))

Accuracy of the network on the 10000 test images: 27 %
Finished testing in 409476.143 seconds
